## 환경설정

* **세부 요구사항**
    - 경로 설정 : Google Colaboratory의 가상 컴퓨터 안에 데이터셋을 다운로드 받고 압축을 해제하세요.
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
        * 필요하다고 판단되는 라이브러리를 추가하세요.

In [1]:
!pip install keras --upgrade

In [2]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install roboflow

## 쿨루프 시공 대상 여부 분류
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습을 진행하세요.
        - 인공위성 지도를 이용하여 이미지를 **추가적으로** 10장 수집하고 추론 과정에서 사용하세요.

### 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [5]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

In [6]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist() # zip_ref에 포함된 모든 파일과 폴더의 이름 목록을 file_list에 저장

        if os.path.exists(f'{file_name[:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') : # 파일 목록 반복하면서 파일 추출. tqdm은 진행률 표시줄로, 각 파일이 추출될 때마다 Extracting과 함께 파일 수 표시
                zip_ref.extract(member=f, path=f'{file_name[:-4]}/')

In [7]:
dataset_extract('/content/drive/MyDrive/KT Aivle/Mini Project 4-2/cool_roof_images.zip')

데이터셋 폴더가 이미 존재합니다.


In [8]:
dataset_extract('/content/drive/MyDrive/KT Aivle/Mini Project 4-2/cool_roof_yolo_labels.zip')

데이터셋 폴더가 이미 존재합니다.


In [9]:
dataset_extract('/content/drive/MyDrive/KT Aivle/Mini Project 4-2/roboflow_cool_roof_dataset.zip')

데이터셋 폴더가 이미 존재합니다.


In [ ]:
dataset_extract('/content/drive/MyDrive/KT Aivle/Mini Project 4-2/roboflow_cool_roof_dataset_t.zip')

https://universe.roboflow.com/ktaivle5/cool_roof_project/dataset/4

https://universe.roboflow.com/start-x6z0m/test_test_test_test/dataset/1

### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

In [10]:
# 데이터셋 경로 설정
dataset_path = '/content/datasets/cool_roof/'
image_train_path = dataset_path + 'images/train'
image_val_path = dataset_path + 'images/val'
label_train_path = dataset_path + 'labels/train'
label_val_path = dataset_path + 'labels/val'

In [11]:
# 디렉토리 생성
os.makedirs(image_train_path, exist_ok=True)
os.makedirs(image_val_path, exist_ok=True)
os.makedirs(label_train_path, exist_ok=True)
os.makedirs(label_val_path, exist_ok=True)

### 데이터 스플릿 & 파일 이동
1. 재현을 위한 난수 고정 : 2024
2. 테스트셋 데이터 20%
- 위 설정에 맞게 데이터를 나누고, 위 과정에서 생성한 폴더에 이동시키세요.

In [12]:
image_path = glob.glob('/content/drive/MyDrive/KT Aivle/Mini Project 4-2/cool_roof_images/*.jpg') # glob: 조건에 맞는 파일명을 리스트 형식으로 반환
label_path = glob.glob('/content/drive/MyDrive/KT Aivle/Mini Project 4-2/cool_roof_yolo_labels/obj_train_data/*.txt')
image_path = sorted(image_path) # glob은 랜덤순으로 가져옴
label_path = sorted(label_path)

In [13]:
image_path2 = glob.glob('/content/drive/MyDrive/KT Aivle/Mini Project 4-2/roboflow_cool_roof_dataset/train/images/*.jpg') # glob: 조건에 맞는 파일명을 리스트 형식으로 반환
label_path2 = glob.glob('/content/drive/MyDrive/KT Aivle/Mini Project 4-2/roboflow_cool_roof_dataset/train/labels/*.txt')
image_path.extend(sorted(image_path2)) # glob은 랜덤순으로 가져옴
label_path.extend(sorted(label_path2))

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_images, val_images, train_labels, val_labels = train_test_split(image_path, label_path, test_size=0.2, random_state=2024)

In [19]:
len(val_labels)

173

In [20]:
# 각 train, val 폴더로 이미지 복사
for image_file in train_images:
    shutil.copy(src = image_file, dst = image_train_path)

for image_file in val_images:
    shutil.copy(src = image_file, dst = image_val_path)

In [21]:
# 각 train, val 폴더로 레이블 복사
for label_file in train_labels:
    shutil.copy(src = label_file, dst = label_train_path)

for label_file in val_labels:
    shutil.copy(src = label_file, dst = label_val_path)

In [22]:
train_images = os.listdir(image_train_path)
val_images = os.listdir(image_val_path)

In [60]:
len(train_images), len(val_images), len(train_labels), len(val_labels)

(690, 173, 690, 173)

### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [24]:
with open('/content/drive/MyDrive/KT Aivle/Mini Project 4-2/cool_roof_yolo_labels/obj.names') as f:
    strings = f.readlines() # 파일에서 모든 줄을 읽어와 저장
    cool_roof, generic_roof = strings[0].strip(), strings[1].strip() # 양끝의 공백과 줄바꿈 문자 제거

roof = {0: cool_roof, 1: generic_roof}

In [25]:
roof

{0: 'cool roof', 1: 'generic roof'}

In [26]:
yaml_dict = {
             'train': './images/train',
             'val': './images/val',
             'names': roof
             }

yaml_dict

{'train': './images/train',
 'val': './images/val',
 'names': {0: 'cool roof', 1: 'generic roof'}}

In [27]:
import yaml

In [28]:
with open('/content/datasets/cool_roof/roof_info.yaml', 'w') as f :
    yaml.dump(yaml_dict, f)

### YOLO v8 모델
- yaml 파일의 경로 설정에 주의하세요.

In [29]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 9.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [30]:
from ultralytics import YOLO, settings

In [31]:
settings
# datasets_dir: /content/datasets 안에 dataset을 다운받겠다는 의미
# trainset의 경로, validset의 경로 등을 설정 가능
# clearml부터 wandb까지는 모델 학습 뒤 결과물 공유하기 위해 저장할 수 있게끔 해둔 것

{'settings_version': '0.0.4',
 'datasets_dir': '/content/datasets',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [32]:
# settings['datasets_dir'] = '/content/'
# settings.update()
# settings

In [33]:
model = YOLO(model='yolov8n.pt', task='detect') # content 폴더에 모델 담음

100%|██████████| 6.23M/6.23M [00:00<00:00, 117MB/s]


- Parameters
    1. data : 학습시킬 데이터셋의 경로. default 'coco128.yaml'
    2. epochs : 학습 데이터 전체를 총 몇 번씩 학습시킬 것인지 설정. default 100
    3. patience : 학습 과정에서 성능 개선이 발생하지 않을 때 몇 epoch 더 지켜볼 것인지 설정. default 50
    4. batch : 미니 배치의 사이즈 설정. default 16. -1일 경우 자동 설정.
    5. imgsz : 입력 이미지의 크기. default 640 (640x640. 비율 맞추기 위해 회색부분 추가)
    6. save : 학습 과정을 저장할 것인지 설정. default True
    7. project : 학습 과정이 저장되는 폴더의 이름.
    8. name : project 내부에 생성되는 폴더의 이름.
    9. exist_ok : 동일한 이름의 폴더가 있을 때 덮어씌울 것인지 설정. default False
    10. pretrained : 사전 학습된 모델을 사용할 것인지 설정. default False
    11. optimizer : 경사 하강법의 세부 방법 설정. default 'auto'
    12. verbose : 학습 과정을 상세하게 출력할 것인지 설정. default False
    13. seed : 재현성을 위한 난수 설정
    14. resume : 마지막 학습부터 다시 학습할 것인지 설정. default False
    15. freeze : 첫 레이어부터 몇 레이어까지 기존 가중치를 유지할 것인지 설정. default None

In [34]:
# Augmentation: https://docs.ultralytics.com/usage/cfg/#augmentation-settings
model.train(data='/content/datasets/cool_roof/roof_info.yaml',
            epochs=100,
            patience=30, # 좀 줄이자
            save=True,
            # project='trained',
            # name='trained_model',
            exist_ok=False,
            pretrained=True,
            optimizer='auto',
            verbose=False,
            seed=2024,
            resume=False,
            freeze=None
            )

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/datasets/cool_roof/roof_info.yaml, epochs=100, time=None, patience=30, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, s

100%|██████████| 755k/755k [00:00<00:00, 23.6MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/datasets/cool_roof/labels/train... 690 images, 0 backgrounds, 0 corrupt: 100%|██████████| 690/690 [00:02<00:00, 275.59it/s]

train: New cache created: /content/datasets/cool_roof/labels/train.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 2066, len(boxes) = 3603. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

                   all        173        919      0.908      0.796       0.86      0.683



100 epochs completed in 0.677 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:08<00:00,  1.47s/it]


                   all        173        919      0.907      0.795       0.86      0.683
Speed: 0.4ms preprocess, 3.4ms inference, 0.0ms loss, 9.5ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a712285a110>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

- Parameters
    1. source : 예측 대상 이미지/동영상의 경로
    2. conf : confidence score threshold. default 0.25 → 더 크게 주면 box더 조금 그려짐
    3. iou : NMS에 적용되는 IoU threshold. default 0.7. threshold를 넘기면 같은 object를 가리키는 거라고 판단.
    4. save : 예측된 이미지/동영상을 저장할 것인지 설정. default False
    5. save_txt : Annotation 정보도 함께 저장할 것인지 설정. default False
    6. save_conf : Annotation 정보 맨 끝에 Confidence Score도 추가할 것인지 설정. default False
    7. line_width : 그려지는 박스의 두께 설정. default None

In [48]:
results = model.predict(source='/content/datasets/cool_roof/images/test',
                        conf=0.25,
                        iou=0.3,
                        save=True,
                        save_txt=True,
                        line_width=2)

Results saved to runs/detect/train5
15 labels saved to runs/detect/train5/labels


In [56]:
model2 = YOLO(model='/content/runs/detect/train/weights/best.pt', task='detect') # content 폴더에 모델 담음

In [62]:
# Augmentation: https://docs.ultralytics.com/usage/cfg/#augmentation-settings
model2.train(data='/content/datasets/cool_roof/roof_info.yaml',
            epochs=5,
            patience=3, # 좀 줄이자
            save=True,
            # project='trained',
            # name='trained_model',
            exist_ok=False,
            pretrained=True,
            optimizer='auto',
            verbose=True,
            seed=2024,
            resume=False,
            freeze=None
            )

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/runs/detect/train/weights/best.pt, data=/content/datasets/cool_roof/roof_info.yaml, epochs=5, time=None, patience=3, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train64, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

train: Scanning /content/datasets/cool_roof/labels/train.cache... 690 images, 0 backgrounds, 0 corrupt: 100%|██████████| 690/690 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 2066, len(boxes) = 3603. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/datasets/cool_roof/labels/val.cache... 173 images, 0 backgrounds, 0 corrupt: 100%|██████████| 173/173 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 520, len(boxes) = 919. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

                   all        173        919      0.838      0.807      0.852      0.666
EarlyStopping: Training stopped early as no improvement observed in last 3 epochs. Best results observed at epoch 1, best model saved as best.pt.
To update EarlyStopping(patience=3) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



4 epochs completed in 0.029 hours.
Optimizer stripped from runs/detect/train64/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train64/weights/best.pt, 6.2MB

Validating runs/detect/train64/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:08<00:00,  1.41s/it]


                   all        173        919      0.886      0.794      0.852      0.668
             cool roof        173        302      0.839      0.874       0.85      0.698
          generic roof        173        617      0.933      0.713      0.853      0.638
Speed: 0.8ms preprocess, 3.4ms inference, 0.0ms loss, 8.5ms postprocess per image
Results saved to runs/detect/train64


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a710c11c2b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [57]:
# Augmentation: https://docs.ultralytics.com/usage/cfg/#augmentation-settings
model2.train(data='/content/datasets/cool_roof/roof_info.yaml',
            epochs=100,
            patience=20, # 좀 줄이자
            save=True,
            # project='trained',
            # name='trained_model',
            exist_ok=False,
            pretrained=True,
            optimizer='auto',
            verbose=False,
            seed=2024,
            resume=False,
            freeze=None
            )

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/runs/detect/train/weights/best.pt, data=/content/datasets/cool_roof/roof_info.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sav

train: Scanning /content/datasets/cool_roof/labels/train.cache... 690 images, 0 backgrounds, 0 corrupt: 100%|██████████| 690/690 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 2066, len(boxes) = 3603. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/datasets/cool_roof/labels/val.cache... 173 images, 0 backgrounds, 0 corrupt: 100%|██████████| 173/173 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 520, len(boxes) = 919. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

                   all        173        919      0.825      0.703      0.791      0.585


EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 1, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

21 epochs completed in 0.147 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 6.2MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:11<00:00,  1.86s/it]


                   all        173        919      0.892      0.777      0.866      0.664
Speed: 0.5ms preprocess, 7.8ms inference, 0.0ms loss, 10.8ms postprocess per image
Results saved to runs/detect/train6


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a712551d990>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [58]:
results = model2.predict(source='/content/datasets/cool_roof/images/test',
                        conf=0.25,
                        iou=0.3,
                        save=True,
                        save_txt=True,
                        line_width=2)

Results saved to runs/detect/train62
15 labels saved to runs/detect/train62/labels


---
Augmentation

In [55]:
a_model = YOLO(model='yolov8n.pt', task='detect') # content 폴더에 모델 담음

In [39]:
# Augmentation: https://docs.ultralytics.com/usage/cfg/#augmentation-settings
a_model.train(data='/content/datasets/cool_roof/roof_info.yaml',
            epochs=1000,
            patience=30,
            save=True,
            # project='trained',
            # name='trained_model',
            exist_ok=False,
            pretrained=True,
            optimizer='auto',
            verbose=False,
            seed=2024,
            resume=False,
            freeze=None,
            augment = True,
            translate= 0.1,
            scale = 0,
            flipud = 0.5, # Flips the image upside down with the specified probability, increasing the data variability without affecting the object's characteristics.
            fliplr = 0.5, # Flips the image left to right with the specified probability, useful for learning symmetrical objects and increasing dataset diversity.
            mosaic = 1.0,
            #shear = 0,
            #perspective = 0,
            #hsv_h = 0.015, # image HSV-Hue
            #hsv_s = 0.7, # image HSV-Saturation
            #hsv_v = 0.4, # image HSV-Value
            #degrees =
            #mixup =
            #copy_paste =
            #auto_augment =
            #erasing =
            #crop_fraction =
            )

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/datasets/cool_roof/roof_info.yaml, epochs=1000, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 

train: Scanning /content/datasets/cool_roof/labels/train.cache... 690 images, 0 backgrounds, 0 corrupt: 100%|██████████| 690/690 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 2066, len(boxes) = 3603. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/datasets/cool_roof/labels/val.cache... 173 images, 0 backgrounds, 0 corrupt: 100%|██████████| 173/173 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 520, len(boxes) = 919. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

                   all        173        919      0.906      0.815      0.865      0.724


EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 230, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

250 epochs completed in 1.718 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.3MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:11<00:00,  1.99s/it]


                   all        173        919      0.768      0.775      0.816      0.603
Speed: 0.6ms preprocess, 21.4ms inference, 0.0ms loss, 11.8ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a7127735630>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [49]:
results = a_model.predict(source='/content/datasets/cool_roof/images/test',
                        conf=0.25,
                        iou=0.3,
                        save=True,
                        save_txt=True,
                        line_width=2)

Results saved to runs/detect/train35
14 labels saved to runs/detect/train35/labels


In [50]:
import matplotlib.pyplot as plt
import os
from PIL import Image

def display_images_in_grid(image_dir):
    # 이미지 파일 목록
    image_files = [file for file in os.listdir(image_dir) if file.endswith('.jpg') or file.endswith('.png')]
    num_images = len(image_files)

    # 이미지를 한 줄에 3개씩 표시
    rows = (num_images + 2) // 3  # 필요한 행 수 계산
    fig, axs = plt.subplots(rows, 3, figsize=(15, 5 * rows))  # subplot 그리드 생성

    for i, image_file in enumerate(image_files):
        img_path = os.path.join(image_dir, image_file)
        img = Image.open(img_path)
        ax = axs[i // 3, i % 3]  # 현재 이미지의 위치를 계산 (행, 열)
        ax.imshow(img)
        ax.set_title(image_file)
        ax.axis('off')  # 축을 표시하지 않음

    # 그리드에 빈 공간이 있으면 숨김
    for j in range(i + 1, rows * 3):
        axs[j // 3, j % 3].axis('off')

    plt.tight_layout()
    plt.show()


In [51]:
image_directory = '/content/runs/detect/train5'
display_images_in_grid(image_directory)

Output hidden; open in https://colab.research.google.com to view.

In [52]:
image_directory = '/content/runs/detect/train35'
display_images_in_grid(image_directory)

Output hidden; open in https://colab.research.google.com to view.

In [59]:
image_directory = '/content/runs/detect/train62'
display_images_in_grid(image_directory)

Output hidden; open in https://colab.research.google.com to view.